# Multithreading in Python

Multithreading allows running multiple threads (lightweight processes) concurrently within a single process. Python uses threads for concurrent execution while sharing the same memory space.

### Basic Threading Example



In [ ]:
import threading
import time

def worker_thread(thread_id):
    # This function will run in a separate thread
    print(f"Thread {thread_id} starting")
    # Simulate some work
    time.sleep(2)
    print(f"Thread {thread_id} finished")

def main():
    # Create a list to store threads
    threads = []
    
    # Create and start 3 threads
    for i in range(3):
        # Create a new thread targeting worker_thread function
        thread = threading.Thread(target=worker_thread, args=(i,))
        threads.append(thread)
        # Start the thread
        thread.start()
    
    # Wait for all threads to complete
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()



### Thread with a Class



In [ ]:
import threading
import time

class WorkerThread(threading.Thread):
    def __init__(self, thread_id):
        # Initialize the thread
        super().__init__()
        self.thread_id = thread_id
    
    def run(self):
        # This method is called when the thread starts
        print(f"Thread {self.thread_id} is starting")
        # Simulate work
        time.sleep(2)
        print(f"Thread {self.thread_id} is finished")

def main():
    # Create and start 3 worker threads
    threads = []
    for i in range(3):
        thread = WorkerThread(i)
        threads.append(thread)
        thread.start()
    
    # Wait for all threads to complete
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()



### Thread Synchronization with Lock



In [ ]:
import threading
import time

class BankAccount:
    def __init__(self):
        self.balance = 0
        # Create a lock for thread synchronization
        self.lock = threading.Lock()
    
    def deposit(self, amount):
        # Acquire the lock before updating balance
        with self.lock:
            # Critical section - only one thread can execute this at a time
            current_balance = self.balance
            time.sleep(0.1)  # Simulate some processing time
            self.balance = current_balance + amount
            print(f"Deposited {amount}. New balance: {self.balance}")

def make_deposits(account):
    # Make 5 deposits of 100 each
    for _ in range(5):
        account.deposit(100)

def main():
    # Create a shared bank account
    account = BankAccount()
    
    # Create two threads that will deposit money
    thread1 = threading.Thread(target=make_deposits, args=(account,))
    thread2 = threading.Thread(target=make_deposits, args=(account,))
    
    # Start both threads
    thread1.start()
    thread2.start()
    
    # Wait for both threads to complete
    thread1.join()
    thread2.join()
    
    print(f"Final balance: {account.balance}")

if __name__ == "__main__":
    main()



### Thread Communication with Event



In [ ]:
import threading
import time

def producer(event):
    # Simulate producing data
    print("Producer: Working on data...")
    time.sleep(2)
    
    # Signal that data is ready
    print("Producer: Data is ready")
    event.set()

def consumer(event):
    # Wait for data to be ready
    print("Consumer: Waiting for data...")
    event.wait()
    
    # Process the data once it's ready
    print("Consumer: Processing data...")

def main():
    # Create an event object
    data_ready = threading.Event()
    
    # Create producer and consumer threads
    prod = threading.Thread(target=producer, args=(data_ready,))
    cons = threading.Thread(target=consumer, args=(data_ready,))
    
    # Start both threads
    prod.start()
    cons.start()
    
    # Wait for both threads to complete
    prod.join()
    cons.join()

if __name__ == "__main__":
    main()



### Thread Pool with Queue



In [ ]:
import threading
import queue
import time

class WorkerThread(threading.Thread):
    def __init__(self, queue):
        super().__init__()
        self.queue = queue
        self.daemon = True  # Thread will exit when main program exits
    
    def run(self):
        while True:
            try:
                # Get task from queue
                task = self.queue.get(timeout=1)
                # Process the task
                print(f"Processing task: {task}")
                time.sleep(0.5)
                # Mark task as done
                self.queue.task_done()
            except queue.Empty:
                break

def main():
    # Create a queue to hold tasks
    task_queue = queue.Queue()
    
    # Create worker threads
    threads = []
    for _ in range(3):
        thread = WorkerThread(task_queue)
        thread.start()
        threads.append(thread)
    
    # Add tasks to the queue
    for i in range(10):
        task_queue.put(f"Task {i}")
    
    # Wait for all tasks to complete
    task_queue.join()
    
    # Wait for all threads to complete
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()



### Key Points to Remember:

1. **Global Interpreter Lock (GIL)**:
   - Python's GIL means threads can't run truly parallel for CPU-bound tasks
   - Best for I/O-bound operations (network, file operations)

2. **Thread Safety**:
   - Use `Lock` for thread synchronization
   - Avoid sharing mutable state between threads
   - Use thread-safe data structures from `queue` module

3. **Thread Communication**:
   - Use `Event` for signaling between threads
   - Use `Queue` for passing data between threads
   - Use `Condition` for complex synchronization

4. **Best Practices**:
   - Always join threads before program exit
   - Use daemon threads for background tasks
   - Handle exceptions within threads
   - Use thread pools for managing multiple workers

5. **When to Use Threading**:
   - I/O-bound tasks (file operations, network calls)
   - GUI applications
   - Concurrent tasks that don't require true parallelism

Remember that due to Python's GIL, multithreading is most effective for I/O-bound tasks. For CPU-bound tasks, consider using multiprocessing instead.